In [1]:
import os
import logging
from tqdm import tqdm, trange

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup
from seqeval.metrics import precision_score, recall_score, f1_score

%cd ../

from bert_finetune_cls.utils import MODEL_CLASSES, compute_metrics, get_intent_labels

logger = logging.getLogger(__name__)

C:\Users\威威的小荔枝\Desktop\第五课_代码


In [2]:
# 计算评价指标

def compute_metrics(intent_preds, intent_labels):
    """
        计算metrics
    """
    assert len(intent_preds) == len(intent_labels)
    results = {}
    intent_result = get_intent_acc(intent_preds, intent_labels)

    results.update(intent_result)

    return results

def get_intent_acc(preds, labels):
    acc = (preds == labels).mean()
    return {
        "intent_acc": acc
    }

In [3]:



class Trainer(object):
    def __init__(self, args, train_dataset=None, dev_dataset=None, test_dataset=None):
        self.args = args
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset
        
        # 加载模型，标签名称到编号的映射 (label maps):
        self.intent_label_lst = get_intent_labels(args)
        # Use cross entropy ignore index as padding label id so that only real label ids contribute to the loss later
        
        # 加载模型的config，model本身
        self.config_class, self.model_class, _ = MODEL_CLASSES[args.model_type]
        self.config = self.config_class.from_pretrained(args.model_name_or_path, finetuning_task=args.task)
        self.model = self.model_class.from_pretrained(args.model_name_or_path,
                                                      config=self.config,
                                                      args=args,
                                                      intent_label_lst=self.intent_label_lst,)
        
        # 将模型放到GPU，如果有的话
        # GPU or CPU
        self.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
        self.model.to(self.device)

    def train(self):
        
        # 加载训练数据
        train_sampler = RandomSampler(self.train_dataset)
        train_dataloader = DataLoader(self.train_dataset, sampler=train_sampler, batch_size=self.args.train_batch_size)
        
        # 计算训练的总的更新步数，用于learning rate的schedule (不是迭代步数)
        if self.args.max_steps > 0:
            t_total = self.args.max_steps
            self.args.num_train_epochs = self.args.max_steps // (len(train_dataloader) // self.args.gradient_accumulation_steps) + 1
        else:
            t_total = len(train_dataloader) // self.args.gradient_accumulation_steps * self.args.num_train_epochs
        
        # 打印一下参数，看看都有哪些
        for n, p in self.model.named_parameters():
            print(n)

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']  # bias和归一化操作中的参数是做weight decay; 
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': self.args.weight_decay},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=self.args.warmup_steps, num_training_steps=t_total)

        # Train!
        logger.info("***** Running training *****")
        logger.info("  Num examples = %d", len(self.train_dataset))
        logger.info("  Num Epochs = %d", self.args.num_train_epochs)
        logger.info("  Total train batch size = %d", self.args.train_batch_size)
        logger.info("  Gradient Accumulation steps = %d", self.args.gradient_accumulation_steps)
        logger.info("  Total optimization steps = %d", t_total)
        logger.info("  Logging steps = %d", self.args.logging_steps)   # 计算dev performance；
        logger.info("  Save steps = %d", self.args.save_steps)         # 保存model checkpoint；

        global_step = 0
        tr_loss = 0.0
        self.model.zero_grad()  # 清空梯度；

        train_iterator = trange(int(self.args.num_train_epochs), desc="Epoch")

        for _ in train_iterator:
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                self.model.train()
                batch = tuple(t.to(self.device) for t in batch)  # 将数据传到设备上面：GPU or CPU

                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'intent_label_ids': batch[3],
                         }
                if self.args.model_type != 'distilbert':
                    inputs['token_type_ids'] = batch[2]
                outputs = self.model(**inputs)
                loss = outputs[0]

                if self.args.gradient_accumulation_steps > 1:
                    loss = loss / self.args.gradient_accumulation_steps

                loss.backward()   # 求梯度

                tr_loss += loss.item()
                if (step + 1) % self.args.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)  # 

                    optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    self.model.zero_grad()  # 清空梯度；
                    global_step += 1

                    if self.args.logging_steps > 0 and global_step % self.args.logging_steps == 0:
                        self.evaluate("dev")

                    if self.args.save_steps > 0 and global_step % self.args.save_steps == 0:
                        self.save_model()

                if 0 < self.args.max_steps < global_step:
                    epoch_iterator.close()
                    break

            if 0 < self.args.max_steps < global_step:
                train_iterator.close()
                break

        return global_step, tr_loss / global_step

    def evaluate(self, mode):
        if mode == 'test':
            dataset = self.test_dataset
        elif mode == 'dev':
            dataset = self.dev_dataset
        else:
            raise Exception("Only dev and test dataset available")

        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

        # Eval!
        logger.info("***** Running evaluation on %s dataset *****", mode)
        logger.info("  Num examples = %d", len(dataset))
        logger.info("  Batch size = %d", self.args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0
        intent_preds = None
        out_intent_label_ids = None

        self.model.eval()

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'intent_label_ids': batch[3],
                         }
                if self.args.model_type != 'distilbert':
                    inputs['token_type_ids'] = batch[2]
                outputs = self.model(**inputs)
                tmp_eval_loss, intent_logits = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1

            # Intent prediction
            if intent_preds is None:
                intent_preds = intent_logits.detach().cpu().numpy()
                out_intent_label_ids = inputs['intent_label_ids'].detach().cpu().numpy()
            else:
                intent_preds = np.append(intent_preds, intent_logits.detach().cpu().numpy(), axis=0)
                out_intent_label_ids = np.append(
                    out_intent_label_ids, inputs['intent_label_ids'].detach().cpu().numpy(), axis=0)

            
        eval_loss = eval_loss / nb_eval_steps
        results = {
            "loss": eval_loss
        }

        # Intent result
        intent_preds = np.argmax(intent_preds, axis=1)

        total_result = compute_metrics(intent_preds, out_intent_label_ids)
        results.update(total_result)

        logger.info("***** Eval results *****")
        for key in sorted(results.keys()):
            logger.info("  %s = %s", key, str(results[key]))

        return results

    def save_model(self):
        # Save model checkpoint (Overwrite)
        if not os.path.exists(self.args.model_dir):
            os.makedirs(self.args.model_dir)
        model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        model_to_save.save_pretrained(self.args.model_dir)

        # Save training arguments together with the trained model
        torch.save(self.args, os.path.join(self.args.model_dir, 'training_args.bin'))
        logger.info("Saving model checkpoint to %s", self.args.model_dir)

    def load_model(self):
        # Check whether model exists
        if not os.path.exists(self.args.model_dir):
            raise Exception("Model doesn't exists! Train first!")

        try:
            self.model = self.model_class.from_pretrained(self.args.model_dir,
                                                          args=self.args,
                                                          intent_label_lst=self.intent_label_lst)
            self.model.to(self.device)
            logger.info("***** Model Loaded *****")
        except:
            raise Exception("Some model files might be missing...")

### 举例查看

In [4]:
from transformers import BertConfig, DistilBertConfig, AlbertConfig
from transformers import BertTokenizer, DistilBertTokenizer, AlbertTokenizer

from bert_finetune_cls.model import ClsBERT
from bert_finetune_cls.utils import init_logger, load_tokenizer, get_intent_labels, set_seed
from bert_finetune_cls.data_loader import load_and_cache_examples

MODEL_CLASSES = {
    'bert': (BertConfig, ClsBERT, BertTokenizer),
}

MODEL_PATH_MAP = {
    'bert': './bert_finetune_cls/resources/uncased_L-2_H-128_A-2',
}

# 先构建参数
class Args():
    task =  None
    data_dir =  None
    intent_label_file =  None


args = Args()
args.seed = 1991
args.no_cuda = True
args.task = "atis"
args.data_dir = "./bert_finetune_cls/data"
args.intent_label_file = "intent_label.txt"
args.max_seq_len = 50
args.model_type = "bert"
args.model_dir = "bert_finetune_cls/experiments/outputs/clsbert_0"
args.model_name_or_path = MODEL_PATH_MAP[args.model_type]

args.train_batch_size = 8
args.eval_batch_size = 16
args.dropout_rate = 0.1

args.max_steps = 1000
args.num_train_epochs = 1
args.gradient_accumulation_steps = 1
args.weight_decay = 1e-5
args.learning_rate = 1e-5
args.adam_epsilon = 1e-8
args.max_grad_norm = 1.0
args.warmup_steps = 100

args.logging_steps = 100
args.save_steps = 200

# 设置随机种子
set_seed(args)

# 加载tokenizer
tokenizer = load_tokenizer(args)

# 加载数据集
train_dataset = load_and_cache_examples(args, tokenizer, mode="train")
dev_dataset = load_and_cache_examples(args, tokenizer, mode="dev")
test_dataset = load_and_cache_examples(args, tokenizer, mode="test")

# 加载trainer
trainer = Trainer(args, train_dataset, dev_dataset, test_dataset)


Some weights of the model checkpoint at ./bert_finetune_cls/resources/uncased_L-2_H-128_A-2 were not used when initializing ClsBERT: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing ClsBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ClsBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ClsBERT were not initialized from the model checkpoint at ./bert_finetune_cls/r

In [5]:
# 训练
trainer.train()

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

Iteration:  16%|██████████████████▌                                                                                               | 91/560 [00:08<00:44, 10.52it/s]


Iteration:  18%|████████████████████▏                                                                                             | 99/560 [00:09<00:43, 10.55it/s]

Evaluating:   0%|                                                                                                                           | 0/32 [00:00<?, ?it/s]

Evaluating:  19%|█████████████████████▌                                                                                             | 6/32 [00:00<00:00, 54.05it/s]

Evaluating:  38%|██████████████████████████████████████████▊                                                                       | 12/32 [00:00<00:00, 53.91it/s]

Evaluating:  59%|███████████████████████████████████████████████████████████████████▋                                              | 19/32 [00:00<00:00, 55.96it/s]

Evaluating:  78%|█████████████████████████████████████████████████████████████████████████████████████████                         | 25/32 [00:00<00:00, 55.37it/s]

Evaluating

Iteration:  35%|███████████████████████████████████████▉                                                                         | 198/560 [00:18<00:31, 11.68it/s]

Evaluating:   0%|                                                                                                                           | 0/32 [00:00<?, ?it/s]

Evaluating:  22%|█████████████████████████▏                                                                                         | 7/32 [00:00<00:00, 62.50it/s]

Evaluating:  44%|█████████████████████████████████████████████████▉                                                                | 14/32 [00:00<00:00, 62.17it/s]

Evaluating:  66%|██████████████████████████████████████████████████████████████████████████▊                                       | 21/32 [00:00<00:00, 62.27it/s]

Evaluating:  88%|███████████████████████████████████████████████████████████████████████████████████████████████████▊              | 28/32 [00:00<00:00, 62.01it/s]

Iteration:

Iteration:  53%|████████████████████████████████████████████████████████████▏                                                    | 298/560 [00:28<00:25, 10.25it/s]

Evaluating:   0%|                                                                                                                           | 0/32 [00:00<?, ?it/s]

Evaluating:  19%|█████████████████████▌                                                                                             | 6/32 [00:00<00:00, 55.05it/s]

Evaluating:  34%|███████████████████████████████████████▏                                                                          | 11/32 [00:00<00:00, 53.09it/s]

Evaluating:  50%|█████████████████████████████████████████████████████████                                                         | 16/32 [00:00<00:00, 50.39it/s]

Evaluating:  66%|██████████████████████████████████████████████████████████████████████████▊                                       | 21/32 [00:00<00:00, 49.97it/s]

Evaluating

...

Iteration:  43%|████████████████████████████████████████████████▏                                                                | 239/560 [00:24<00:30, 10.55it/s]

Evaluating:   0%|                                                                                                                           | 0/32 [00:00<?, ?it/s]

Evaluating:  16%|█████████████████▉                                                                                                 | 5/32 [00:00<00:00, 45.87it/s]

Evaluating:  28%|████████████████████████████████▎                                                                                  | 9/32 [00:00<00:00, 43.36it/s]

Evaluating:  44%|█████████████████████████████████████████████████▉                                                                | 14/32 [00:00<00:00, 43.63it/s]

Evaluating:  59%|███████████████████████████████████████████████████████████████████▋                                              | 19/32 [00:00<00:00, 42.58it/s]

Evaluating

Iteration:  61%|████████████████████████████████████████████████████████████████████▍                                            | 339/560 [00:35<00:21, 10.18it/s]

Evaluating:   0%|                                                                                                                           | 0/32 [00:00<?, ?it/s]

Evaluating:  19%|█████████████████████▌                                                                                             | 6/32 [00:00<00:00, 58.82it/s]

Evaluating:  34%|███████████████████████████████████████▏                                                                          | 11/32 [00:00<00:00, 55.13it/s]

Evaluating:  50%|█████████████████████████████████████████████████████████                                                         | 16/32 [00:00<00:00, 52.80it/s]

Evaluating:  66%|██████████████████████████████████████████████████████████████████████████▊                                       | 21/32 [00:00<00:00, 51.77it/s]

Evaluating

Iteration:  78%|███████████████████████████████████████████████████████████████████████████████████████▉                         | 436/560 [00:45<00:14,  8.73it/s]


Iteration:  78%|████████████████████████████████████████████████████████████████████████████████████████▌                        | 439/560 [00:46<00:14,  8.07it/s]

Evaluating:   0%|                                                                                                                           | 0/32 [00:00<?, ?it/s]

Evaluating:  16%|█████████████████▉                                                                                                 | 5/32 [00:00<00:00, 46.74it/s]

Evaluating:  28%|████████████████████████████████▎                                                                                  | 9/32 [00:00<00:00, 43.76it/s]

Evaluating:  41%|██████████████████████████████████████████████▎                                                                   | 13/32 [00:00<00:00, 40.61it/s]

Evaluating:  53%|████████████████████████████████████████████████████████████▌                                                     | 17/32 [00:00<00:00, 38.90it/s]

Evaluating

(1001, 2.1433121363718906)

In [7]:
# 评估

trainer.load_model()
trainer.evaluate("dev")


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 64.26it/s]


{'loss': 1.774277739226818, 'intent_acc': 0.714}

In [8]:
trainer.evaluate("test")

Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:00<00:00, 58.21it/s]


{'loss': 1.807569729430335, 'intent_acc': 0.7077267637178052}